<a href="https://colab.research.google.com/github/Chaqmoqjon/Machine-Learning/blob/main/4_Machine_Learning_Full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn

##Datani ajratish
Train va test uchun datani ajratish

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv')
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://images.shiksha.com/mediadata/ugcDocuments/images/wordpressImages/2022_12_MicrosoftTeams-image-6.jpg" alt="" width="600" />'))

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set =  train_test_split(df, test_size= 0.2, random_state  = 42)

In [ ]:
X_train = train_set.drop('median_house_value', axis = 1) #parametrlar: "bashorat" qiluvchilar
y = train_set['median_house_value'].copy() #labels: topishimiz kerak bo'lgan qiymatlar

In [ ]:
housing_num = X_train.drop('ocean_proximity', axis = 1)

##Custom Transformer
rooms_per_household, population_per_household, bedrooms_per_room qiymatlarini olish uchun funksiya

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://miro.medium.com/v2/resize:fit:574/1*hMGRmj9Wz-xGVJhlc08szQ.png" alt="" width="600" />'))

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin #shaxsiy transformer yasash

#bizga kerak ustunlar indekslari
rooms_ix, badrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True):
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y = None):
    return self  #bizning funksiya faqat transformer, estimator emas
  def transform(self, X):
    rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
    population_per_household = X[:, population_ix] / X[:, households_ix]
    if self.add_bedrooms_per_room: #add_bedrooms_per_room ustuni ixtiyoriy bo'ladi
      bedrooms_per_room = X[:, badrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]

##Pipeline
Raqamli ustunlar uchun

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://miro.medium.com/v2/resize:fit:680/1*c4SNMDj18FHQakGS6Gmgsg.png" alt="" width="600" />'))

In [ ]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')), #NaN qiymatlarni median bilan to'ldirildi
    ('attribs_adder', CombinedAttributesAdder()), #Yangi ustunlar qo'shildi
    ('std_scaler', StandardScaler()) #Katta sonlar kichiklashtirildi, standartlashtirildi
])

In [ ]:
num_pipeline.fit_transform(housing_num) #hammasiga birdaniga fit_transform ishlatildi

array([[ 1.27258656, -1.3728112 ,  0.34849025, ..., -0.17491646,
         0.05137609, -0.2117846 ],
       [ 0.70916212, -0.87669601,  1.61811813, ..., -0.40283542,
        -0.11736222,  0.34218528],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.08821601,
        -0.03227969, -0.66165785],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ..., -0.60675918,
         0.02030568,  0.99951387],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.40217517,
         0.00707608, -0.79086209],
       [-1.41489815,  0.99543676,  1.85617335, ..., -0.85144571,
        -0.08535429,  1.69520292]])

##ColumnTransformer
Raqamli va yozuvli ustunlar uchun.

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://miro.medium.com/max/1190/1*I0F-ALOL8J8f6V33CDKyrA.png" alt="" width="600" />'))

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),  #raqamli ustunlar
    ('cat', OneHotEncoder(), cat_attribs) #yozuvli ustunlar
])

In [ ]:
X_prepared = full_pipeline.fit_transform(X_train) #barchasini birdaniga transform qilish

#Linear Regression

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://media.geeksforgeeks.org/wp-content/uploads/20231129130431/11111111.png" alt="" width="600" />'))

In [ ]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression() #mana shu kod bilan...

In [ ]:
LR_model.fit(X_prepared, y) #savol va javob, bilan o'rgatildi

LinearRegression()

In [ ]:
test_data = X_train.sample(10) #mana shu datadan foydalanib...
test_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
15337,-117.31,33.19,11.0,20944.0,3753.0,8738.0,3441.0,4.3762,NEAR OCEAN
5087,-118.28,33.98,45.0,1720.0,416.0,1382.0,365.0,0.9337,<1H OCEAN
7341,-118.14,33.97,29.0,1846.0,530.0,2576.0,528.0,2.6300,<1H OCEAN
17278,-119.74,34.43,26.0,3119.0,562.0,1459.0,562.0,5.0434,NEAR OCEAN
14011,-117.17,32.76,45.0,3149.0,639.0,1160.0,661.0,2.7266,NEAR OCEAN
17795,-121.81,37.39,34.0,2218.0,286.0,827.0,299.0,7.4559,<1H OCEAN
2253,-119.82,36.80,33.0,1670.0,256.0,528.0,250.0,2.9471,INLAND
11459,-117.99,33.68,18.0,2024.0,462.0,1047.0,451.0,3.5848,<1H OCEAN
19882,-119.24,36.33,9.0,3289.0,621.0,1866.0,631.0,3.1599,INLAND
3238,-119.56,36.10,25.0,1093.0,262.0,893.0,252.0,2.1300,INLAND


In [ ]:
test_labels = y.loc[test_data.index] #mana qiymatlarni "bashorat" qilamiz
test_labels

,median_house_value
15337,215500.0
5087,92300.0
7341,156000.0
17278,340400.0
14011,354200.0
17795,456500.0
2253,99500.0
11459,186900.0
19882,95000.0
3238,50800.0


In [ ]:
test_data_prepared = full_pipeline.transform(test_data) #mana shu datani tayyorlaymiz...

In [ ]:
predicted_labels = LR_model.predict(test_data_prepared) #X_train ni ichidagi 10 ta tasodifiy data

In [ ]:
predicted_labels #tahmin qilindan uy narxlari

array([318798.17080017, 111888.16359245, 140204.53191919, 300974.44015637,
       226116.84099373, 369744.66612513, 116424.51104282, 210672.18492743,
        94880.31019436,  86303.41293363])

In [ ]:
pd.DataFrame({'Bashorat': predicted_labels, 'Real': test_labels}) #asl narx bilan solishtirildi

,Bashorat,Real
15337,318798.170800,215500.0
5087,111888.163592,92300.0
7341,140204.531919,156000.0
17278,300974.440156,340400.0
14011,226116.840994,354200.0
17795,369744.666125,456500.0
2253,116424.511043,99500.0
11459,210672.184927,186900.0
19882,94880.310194,95000.0
3238,86303.412934,50800.0


#Modelni baholash

In [ ]:
test_set #test uchun ajratilgan data. 20% lik

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20046,-119.01,36.06,25.0,1505.0,NaN,1392.0,359.0,1.6812,47700.0,INLAND
3024,-119.46,35.14,30.0,2943.0,NaN,1565.0,584.0,2.5313,45800.0,INLAND
15663,-122.44,37.80,52.0,3830.0,NaN,1310.0,963.0,3.4801,500001.0,NEAR BAY
20484,-118.72,34.28,17.0,3051.0,NaN,1705.0,495.0,5.7376,218600.0,<1H OCEAN
9814,-121.93,36.62,34.0,2351.0,NaN,1063.0,428.0,3.7250,278000.0,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...,...
15362,-117.22,33.36,16.0,3165.0,482.0,1351.0,452.0,4.6050,263300.0,<1H OCEAN
16623,-120.83,35.36,28.0,4323.0,886.0,1650.0,705.0,2.7266,266800.0,NEAR OCEAN
18086,-122.05,37.31,25.0,4111.0,538.0,1585.0,568.0,9.2298,500001.0,<1H OCEAN
2144,-119.76,36.77,36.0,2507.0,466.0,1227.0,474.0,2.7850,72300.0,INLAND


In [ ]:
X_test = test_set.drop('median_house_value', axis = 1) #savollar
y_test = test_set['median_house_value'].copy() #javoblar

In [ ]:
X_test_prepared = full_pipeline.transform(X_test) #data to'ldirildi

In [ ]:
y_predicted = LR_model.predict(X_test_prepared) #uy narxni tahmin qilindi

In [ ]:
pd.DataFrame({'Bashorat': y_predicted, 'Real': y_test}) #asl data bilan solishtirildi

,Bashorat,Real
20046,61874.254601,47700.0
3024,121853.525111,45800.0
15663,267770.943681,500001.0
20484,264468.298380,218600.0
9814,258485.695855,278000.0
...,...,...
15362,214774.565911,263300.0
16623,236868.742582,266800.0
18086,447837.046479,500001.0
2144,117275.921461,72300.0


##MAE, RMSE

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error #xatolikni tekshirish uchun funksiya

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://cdn.hackernoon.com/images/GM6DSaJcTeUSvYIRlwBoj4pUrXi1-w4e3hl9.jpeg" alt="" width="600" />'))

In [ ]:
mae = mean_absolute_error(y_test, y_predicted) #asl qiymat, tahmin qilingan qiymat
print(f" MAE: {mae}.  Mashina o'rtacha xatoligi: ~50_898 $")

 MAE: 50898.73953494079.  Mashina o'rtacha xatoligi: ~50_898 $


In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://cdn.hackernoon.com/images/GM6DSaJcTeUSvYIRlwBoj4pUrXi1-u9g3hhm.jpeg" alt="" width="600" />'))

In [ ]:
mse = mean_squared_error(y_test, y_predicted) #asl qiymat, tahmin qilingan qiymat
print(f"RMSE: {np.sqrt(mse)}.  Mashina o'rtacha kvadrat xatoligi: ~72_701 $")

RMSE: 72701.32600762135.  Mashina o'rtacha kvadrat xatoligi: ~72_701 $


#Random Forest

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://www.researchgate.net/publication/313489088/figure/fig3/AS:864415041732616@1583104014685/Fig-A10-Random-Forest-Regressor-The-regressor-used-here-is-formed-of-100-trees-and-the.jpg" alt="" width="600" />'))

In [ ]:
from sklearn.ensemble import RandomForestRegressor #Linear Regression dan yaxshiroq ekan
RF_model = RandomForestRegressor()

In [ ]:
RF_model.fit(X_prepared, y) #o'qitildi

RandomForestRegressor()

In [ ]:
y_predicted = RF_model.predict(X_test_prepared) #20% dagi data. Ishlatib narx tahmin qilindi

In [ ]:
mae = mean_absolute_error(y_test, y_predicted) #xatolik
print(f" MAE: {mae}.  Mashina o'rtacha xatoligi: ~{mae} $")

 MAE: 32398.476533430232.  Mashina o'rtacha xatoligi: ~32398.476533430232 $


In [ ]:
mse = mean_squared_error(y_test, y_predicted) #xatolik
print(f"RMSE: {np.sqrt(mse)}.  Mashina o'rtacha kvadrat xatoligi: ~ {np.sqrt(mse)} $")

RMSE: 50017.921006733966.  Mashina o'rtacha kvadrat xatoligi: ~ 50017.921006733966 $


#Cross Validation

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<img src="https://www.sharpsightlabs.com/wp-content/uploads/2024/02/cross-validation-explained_FEATURED-IMAGE.png" alt="" width="600" />'))

In [ ]:
X = df.drop('median_house_value', axis = 1)
y = df['median_house_value'].copy()

In [ ]:
X_prepared = full_pipeline.fit_transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
mse_scores = cross_val_score(LR_model, X_prepared, y, scoring = 'neg_mean_squared_error', cv = 5) #cv = cross validation

In [ ]:
def display_scores(scores):
  print(f"Scores: {scores} ")
  print(f"Mean: {scores.mean()}")
  print(f"Standard deviation: {scores.std()}")

In [ ]:
display_scores(np.sqrt(-mse_scores))

Scores: [73391.42036892 74809.28332317 75429.91837496 76604.35506436
 66196.72436926] 
Mean: 73286.34030013601
Standard deviation: 3693.161254481324


#Datani *yuklash*

Pickle. 1-usul

In [ ]:
import pickle

In [ ]:
file_name = 'RF_model.pkl' #faylni yuklash
with open(file_name, 'wb') as file:
  pickle.dump(RF_model, file)

In [ ]:
with open(file_name, 'rb') as file: #faylni chaqirish
  RF_model_loaded = pickle.load(file)

Joblib. 2-usul

In [ ]:
import joblib

In [ ]:
file_name = 'LR_model.jbl' #faylni yuklash
joblib.dump(LR_model, file_name)

['LR_model.jbl']

In [ ]:
LR_model_loaded = joblib.load(file_name) #faylni chaqirish

Tekshirildi

In [ ]:
scores = cross_val_score(LR_model_loaded, X_prepared, y, scoring = 'neg_mean_squared_error', cv = 5)
display_scores(np.sqrt(-scores))

Scores: [73391.42036892 74809.28332317 75429.91837496 76604.35506436
 66196.72436926] 
Mean: 73286.34030013601
Standard deviation: 3693.161254481324
